# Project - IFT712

## Data preparation

In [13]:
import data_manager.data_manager
import data_manager.data_organizer

In [37]:
# Loading the database
db = data_manager.data_manager.DatabaseManager('')
db.load()
# Storing the data
dataset = db.get_train_dataset().drop('id', axis=1)

In [15]:
do = data_manager.data_organizer.DataOrganizer()
do.shuffle(dataset, dataset.columns) # Shuffling the data
do.standardize(dataset, dataset.columns[1:]) # Standardization of numeric values
do.numerize(dataset, dataset.columns[0]) # Numerization of the name of each leaf

In [16]:
# Dividing the data in a training and test dataset
training_dataset = dataset[:int(dataset.shape[0] * 0.7)]
test_dataset = dataset[int(dataset.shape[0] * 0.7):]

# Dividing inputs and targets
training_inputs = training_dataset.drop('species', axis=1)
training_targets = training_dataset['species']

test_inputs = test_dataset.drop('species', axis=1)
test_targets = test_dataset['species']

## KNN :

In [17]:
import models.kneighbors_classifier

knn = models.kneighbors_classifier.KneighborsClassifier()

The list of hyperparameters we are looking to optimize : grid_parameters = {'n_neighbors': range(2, 15)}

In [18]:
knn.train(training_inputs, training_targets)

In [19]:
knn.score(test_inputs, test_targets)

0.9595959595959596

In [20]:
print(knn.get_classifier().best_params_)

{'n_neighbors': 2}


## SVM :

In [21]:
import models.svm_classifier
svm = models.svm_classifier.SVMClassifier()

The list of hyperparameters we are looking to optimize : 
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']} 

In [22]:
svm.train(training_inputs, training_targets)

In [23]:
svm.score(test_inputs, test_targets)

0.98989898989899

In [24]:
print(svm.get_classifier().best_params_)

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


## Logistic Regression :

In [25]:
import models.logistic_regression_classifier
lr = models.logistic_regression_classifier.LogisticRegressionClassifier()

The list of hyperparameters we are looking to optimize : param_grid = {'C': [0.1, 1, 10, 100, 1000]}

In [26]:
lr.train(training_inputs, training_targets)

In [27]:
lr.score(test_inputs, test_targets)

0.98989898989899

In [28]:
print(lr.get_classifier().best_params_)

{'C': 1000}


## MLP :

In [29]:
import models.mlp_classifier
mlp = models.mlp_classifier.MLPClassifier()

The list of hyperparameters we are looking to optimize : param_grid = {'alpha': 10.0 ** -np.arange(1, 6), 'hidden_layer_sizes': [(99,), (99, 99), (99, 99, 99)]}

We chose 99 neurons by layer because there are 99 different leaves but it would be interesting to test if other configurations can improve the score.

In [30]:
mlp.train(training_inputs, training_targets)

In [31]:
mlp.score(test_inputs, test_targets)

0.9831649831649831

In [32]:
print(mlp.get_classifier().best_params_)

{'alpha': 0.1, 'hidden_layer_sizes': (99,)}


## Perceptron :

In [33]:
import models.perceptron_classifier
per = models.perceptron_classifier.PerceptronClassifier()

The list of hyperparameters we are looking to optimize : param_grid = {'alpha': 10.0 ** -np.arange(1, 6)}

In [34]:
per.train(training_inputs, training_targets)

In [35]:
per.score(test_inputs, test_targets)

0.8316498316498316

In [36]:
print(per.get_classifier().best_params_)

{'alpha': 0.1}


## SVC :

In [5]:
import models.svc_classifier
svc = models.svc_classifier.SVCClassifier()

The list of hyperparameters we are looking to optimize : param_grid = {'C': [0.1, 1, 10],
                    'loss': ['hinge', 'squared_hinge']}

In [6]:
svc.train(training_inputs, training_targets)

C:\Users\p_bon\Desktop\Sherbrooke\Semestre 1\IFT712\env_TP4\.env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\p_bon\Desktop\Sherbrooke\Semestre 1\IFT712\env_TP4\.env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\p_bon\Desktop\Sherbrooke\Semestre 1\IFT712\env_TP4\.env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\p_bon\Desktop\Sherbrooke\Semestre 1\IFT712\env_TP4\.env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\p_bon\Desktop\Sherbrooke\Semestre 1\IFT712\env_TP4\.env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to co

Même en augmentant le nombre d'itérations maximale, l'algorithme ne parvient pas à converger. Il se peut qu'il ne soit pas adapter au problème mais il serait intéressant de tester l'algorithme avec une grille d'hyperparamètres plus large. On constate tout de même qu'il donne de bons résultats
Even by increasing the number of maximum iterations, the algorithm was not able to converge. SVC might not be adapted to the problem since it is linear. Still, his results 

In [8]:
svc.score(test_inputs, test_targets)

0.9528619528619529

In [12]:
print(svc.get_classifier().best_params_)


{'C': 0.1, 'loss': 'squared_hinge'}
